In [ ]:
# 데이터 구조
import pandas as pd
# 데이터 시각화
import matplotlib.pyplot as plt
import matplotlib
# 나무 구조 시각화
from sklearn.tree import plot_tree
# 데이터 분할:train, test
from sklearn.model_selection import train_test_split
# Random Forest
from sklearn.ensemble import RandomForestClassifier
# 최적 모델, 파라미터 탐색
from sklearn.model_selection import GridSearchCV
# 모델 성능 평가
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [ ]:
# 한글 폰트 설정
matplotlib.rc("font", family = "Malgun Gothic")
# 음수 표시 설정
matplotlib.rc("axes", unicode_minus = False)

In [ ]:
# 데이터 불러오기
df_raw = pd.read_csv("../data/bank.csv")
df_raw.head()

In [ ]:
# 결측치 확인
df_raw.isnull().sum(axis = 0)
# JOB 변수의 결측치는 "Other"로 대체
df_raw["JOB"].fillna("Other", inplace = True)
# 숫자형 변수의 결측치는 해당 변수의 평균값으로 대체
df_raw.fillna(df_raw.mean(), inplace=True)

In [ ]:
# get_dummies: 데이터의 문자형 변수에 대한 더미변수 생성
df_raw_dummy = pd.get_dummies(df_raw)
# 더미변수 생성된 데이터의 상위 5개 row를 확인
df_raw_dummy.head()

In [ ]:
# 학습용/평가용 데이터 분리
df_train, df_test = train_test_split(df_raw_dummy, test_size = 0.3, random_state = 1234)
print("학습용 데이터의 크기 : {}".format(df_train.shape))
print("평가용 데이터의 크기 : {}".format(df_test.shape))

In [ ]:
# 설명변수와 목표변수를 분리
df_train_y = df_train["BAD"]
df_train_x = df_train.drop("BAD", axis = 1, inplace = False)
df_test_y = df_test["BAD"]
df_test_x = df_test.drop("BAD", axis = 1, inplace = False)

In [ ]:
# 랜덤 포레스트 모델 생성
rf_uncustomized = RandomForestClassifier(random_state=1234)
rf_uncustomized.fit(df_train_x, df_train_y)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(rf_uncustomized.score(df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(rf_uncustomized.score(df_test_x, df_test_y)))

In [ ]:
# error 저장 리스트
oob_error = []
for v_n_estimator in range(1, 151):
    rf = RandomForestClassifier(n_estimators = v_n_estimator, oob_score = True, random_state= 1234)
    rf.fit(df_train_x, df_train_y)
    oob_error.append(1 - rf.oob_score_)

In [ ]:
# 그래프로 표현
plt.plot(range(1, 151), oob_error)
plt.ylabel("oob error")
plt.xlabel("n_estimators")
# 트리 수 지정
n_estimators = 100

In [ ]:
estimator = RandomForestClassifier(random_state=1234)
# 구하고자 하는 parameter와 범위
param_grid = {"n_estimators": [100, 150, 200], "max_depth": [4, 5, 6, 7, 8],
"min_samples_leaf": [1, 2, 3, 4, 5]}
# 정확도가 높은 최적 parameter 찾기
grid_rf = GridSearchCV(estimator, param_grid, scoring="accuracy", n_jobs = -1)
grid_rf.fit(df_train_x, df_train_y)
print("best estimator model: \n{}".format(grid_rf.best_estimator_))
print("\nbest parameter: \n{}".format(grid_rf.best_params_))
print("\nbest score: \n{}".format(grid_rf.best_score_.round(3)))

In [ ]:
# 최종 모델
rf_final = RandomForestClassifier(min_samples_leaf = 1, max_depth = 8, n_estimators = 100,
random_state=1234)
rf_final.fit(df_train_x, df_train_y)
# 평가
y_pred = rf_final.predict(df_test_x)
# 학습용 데이터 셋 정확도
print("학습용 데이터의 정확도 : {:.3f}".format(rf_final.score(df_train_x, df_train_y)))
# 평가용 데이터 셋 정확도
print("평가용 데이터의 정확도 : {:.3f}".format(rf_final.score(df_test_x, df_test_y)))
# confusion matrix
print("Confusion matrix: \n{}".format(confusion_matrix(df_test_y, y_pred)))

In [ ]:
# classification Report
print("Classification Report : ", classification_report(df_test_y, y_pred))

In [ ]:
# tree.feature_importances_로 설명변수 중요도 확인 및 테이블로 저장
df_importance = pd.DataFrame()
df_importance ["Feature"] = df_train_x.columns
df_importance ["Importance"] = rf_final.feature_importances_
# df_feature_importance의 테이블을 중요도 순으로 정렬
df_importance .sort_values("Importance", ascending=False, inplace = True)
df_importance .round(3)

In [ ]:
# 설명변수 중요도 그래프
df_importance .sort_values("Importance", ascending=True, inplace = True)
coordinates = range(len(df_importance ))
plt.barh(y = coordinates, width = df_importance ["Importance"])
plt.yticks(coordinates, df_importance ["Feature"])
plt.xlabel("설명변수 중요도")
plt.ylabel("설명변수")